Author: Lester Hedges<br>
Email:&nbsp;&nbsp; lester.hedges@bristol.ac.uk

___Jupyter Recap___:
* Press Shift+Enter to execute a cell and move to the cell below.
* Press Ctrl+Enter to execute a cell and remain in that cell.
* Run a shell command on the underlying operating system by prefixing the command with an exclamation mark, !
* Remember that the flow is in the order that you execute cells, which is not necessarily linear in the notebook. Keep track of the numbers in brackets to the left of the cell!

# Introduction

Welcome to the [CCP-BioSim](http://www.ccpbiosim.ac.uk) workshop on [BioSimSpace](https://biosimspace.org), the EPSRC flagship software project. In this workshop you'll learn how to use BioSimSpace to write robust and portable molecular workflow components and interact directly with running molecular dynamics processes inside of a Jupyter notebook.

As a computational chemist you are likely overwhelmed by the amount of different software packages that are available to you. Having choice is a good thing, but too much can become a burden. I'm sure you have all, at times, come across at least one of the following conundrums:

* How do I know what is the best tool for the job?
* How much time should I invest in learning this new package?
* How can I possibly become an expert in everything?
* How can I share my incredible script with a collaborator who doesn't use the same software.
* Will my workflow break if I upgrade to the latest version of my favourite package?
* Will I get the same result if I re-run my analysis from last month? 

Solving these problems is the core goal of BioSimSpace. The wealth of fantastic software in our community is a real asset but _interoperability_ is currently a problem. Since there is no point reinventing the wheel, BioSimSpace is not an attempt to produce yet another molecular simulation package that reproduces all of the functionality from existing programs. This would result in just another tool for you to learn, along with yet another set of standards and formats. Instead, BioSimSpace is essentially just a set of _shims_, or bits of _glue_, that connect together existing software packages, allowing you to interact with them using a consistent Python interface.

With BioSimSpace you will be able to:

* Write generic workflow components _once_ in a package-agnostic language.
* Run the same script from the command-line, Jupyter, or your within various workflow engines.
* Know that you are running your job using the most suitable package that is availabe on your computer.
* Continue using your favourite package X but be able to share scripts with your collaborator who prefers package Y.
* Publish workflow components online and search our database for others that may be of use to you.
* Be able to take advantage of new software packages and hardware resources as and when they become available.

## Molecular dynamics

One of the core features of BioSimSpace is the ability to set up and run [molecular dynamics](https://en.wikipedia.org/wiki/Molecular_dynamics) (MD) simulations. There are a large number of packages that can run molecular dynamics for biomolecules and BioSimSpace supports several of these: [AMBER](http://ambermd.org), [GROMACS](http://www.gromacs.org), and [NAMD](http://www.ks.uiuc.edu/Research/namd). We also have our own built in GPU accelerated molecular dynamics engine, SOMD, as a fall back in case no other packages are installed. (SOMD is a wrapper around the excellent [OpenMM](http://openmm.org) package.)

While, broadly speaking, the different molecular dynamics engines offer a similar range of features, their interfaces are quite different. This makes it hard to take expertise in one package and immediately apply it to another. At the heart of this problem is the incompatibility between the molecular file formats used by the different packages. While they all contain the same information, i.e. how atoms are laid out in space and how they interact with each other, the structure of the files is very different. In order to provide interoperability betwen packages we will need to be able to read and write many different file formats, and be able to interconvert between them too.

### File conversion

Boring as file parsers are, this is perhaps the most fundamental part of BioSimSpace. Without it, nothing else would work! BioSimSpace can read and write a large number of common molecular file formats, and where possible, can interconvert between many of these too. Let's import the BioSimSpace Python package and see what we can do. For convenience, we'll rename the package to BSS to save us typing:

In [ ]:
import BioSimSpace as BSS

To see what sub-packages are available, simply type `BSS.` in the cell below then hit the Tab key.

In [ ]:
BSS.

Unsurprisingly, the functionality for reading and writing files is contained within the `BSS.IO` packge. To find out more about this packge, type `help(BSS.IO)` in the cell below and run it.

In [ ]:
help(BSS.IO)

BioSimSpace is fully documented, so you can always use `help` on any package, function, class, etc., to find out more information. There is also full documentation on our [website](https://biosimspace.org) if you prefer reading things there. The website is automatically generated from the source code so is always in sync.

Feel free to explore the documentation for any of the functions in the `BSS.IO` package in the cell below.

To see what file formats are supported by BioSimSpace, execute the cell below.

In [ ]:
BSS.IO.fileFormats()

Note that these refer to specific file _formats_, rather than file _extensions_. BioSimSpace doesn't care about file extensions, it's the _contents_ of the file that's important. 

If you aren't familiar with a particular format, you can get more information as follows, e.g.:

In [ ]:
BSS.IO.formatInfo("GroTop")

The `BSS.IO.readMolecules` function is used to read molecular information from file. We've provided some example input files for you in the `input` directory. Let's take a look at some of these.

In [ ]:
!ls input/amber

The files define a solvated alanine dipeptide system in AMBER format. Execute the cell below to see the topology file:

In [ ]:
!head -n 50 input/amber/ala.top

Let's now read the molecules from file. The `BSS.IO.readMolecules` function can [glob](https://en.wikipedia.org/wiki/Glob_(programming)) to perform pattern matching using wildcard characters. (Here the asterisk matches any characters, i.e. we are reading _all_ files in the `input/amber` directory.)

In [ ]:
system = BSS.IO.readMolecules("input/amber/*")

Note that we don't have to specify anything about the file format. BioSimSpace actually reads the files with _all_ of its inbuilt parsers in parallel. If a parser fails to read the file then it is immediately rejected and we move on to the next. As long as all of the files that are read contain a consistent topology, then BioSimSpace will be able to read them.

To see what file formats are associated with the system, run:

In [ ]:
system.fileFormat()

As expected, BioSimpace has detected that these were AMBER format topology and coordinate files.

The molecules are now loaded into a `System` object. We can query how many molecules there are as follows:

In [ ]:
system.nMolecules()

To see how many water molecules there are:

In [ ]:
system.nWaterMolecules()

As always, you can simply type `help(system)` to find out what other options are available.

(The BioSimSpace `System` is simply are thin wrapper around the system class from [Sire](https://github.com/michellab/Sire). For those that are familiar with Sire, you can get access to the underlying object directly using `system._getSireSystem()`. This _private_ method is hidden from the user.)

Now that we have a molecular system, let's write it back to disk in a different format. Unsurprisingly, this is done using the `BSS.IO.saveMolecules` function. Execute the cell below to write the system to GROMACS format coordinate and topology files.

In [ ]:
BSS.IO.saveMolecules("ala", system, ["Gro87", "GroTop"])

Run the cell below to examine the GROMACS topology file.

In [ ]:
!head -n 50 ala.top

Hurrah, we have successfully converted from AMBER to GROMACS format input files!

### Molecular dynamics protocols

Now that we can load molecules, we need to decide what we want to do to them. BioSimSpace provides support for several common molecular simulation _protocols_, which are defined in the `BSS.Protocol` package. Let's look at the help:

In [ ]:
help(BSS.Protocol)

The classes in this package represent high-level descriptions of protocols with a set of options that are shared across the supported molecular dynamics engines. There is often not a lot of configurability, but in most cases the default will work just fine. (As we'll see later, BioSimSpace also allows you to dive in and fine tune things to your specific requirements.)

Use the cell below if you'd like to explore the various protocols in more detail.

In [ ]:
help(BSS.Protocol.Equilibration)

Let's create a `Minimisation` protocol to apply to our `System`. We'll keep it short, so let's specify 1000 steps.

In [ ]:
protocol = BSS.Protocol.Minimisation(steps=1000)

### Molecular dynamics processes

We now have a `System` and a `Protocol`. What we need next is a molecular dynamics _process_ to apply the protocol to the system. Functionality for setting up and running MD processes is contained in the `BSS.Process` package.

In [ ]:
help(BSS.Process)

Let's use the `BSS.Process.Amber` MD driver class to initialise a process to run our simulation with AMBER.

In [ ]:
process = BSS.Process.Amber(system, protocol)

A lot of complexity is hidden in this line. BioSimSpace has automatically found an AMBER executable on the underlying operating system, has automatically written AMBER format molecular input files, generated an AMBER configuration file for the minimisation protocol, and configured any command-line arguments that are required.

By default, processes are run inside of a temporary working directory hidden away from the user. To see where this is, execute the cell below.

In [ ]:
process.workDir()

To see what executable was found, run:

In [ ]:
process.exe()

To see the list of autogenerated input files:

In [ ]:
process.inputFiles()

If you like, we could zip up the input files to use on another occasion. Also, since we're working on a notebook server, let's return a file link so we can download them:

In [ ]:
process.getInput(file_link=True)

We can query also query the list of configuration file options:

In [ ]:
process.getConfig()

And also get command-line argument string for the process:

In [ ]:
process.getArgString()

If you're an expert in a particular package then BioSimSpace allows you to fully customise the process by tweaking the configuration options and command-line arguments. Read the help documentation for `process.setConfig` and `process.setArgs` if you are interested.

Now that we have a process we actually want to run it. Execute the cell below to start the process:

In [ ]:
process.start()

BioSimSpace has now launched a minimisation process in the background! This means you can carry on working in your notebook and periodically check in on the process to see how its doing.

To check whether the process is running:

In [ ]:
process.isRunning()

We can see how many minutes it has been running for:

In [ ]:
process.runTime()

Since this is a short minimisation it will likely finish pretty quickly. Let's print the final energy of the system and return the minimised molecular configuration.

In [ ]:
print(process.getTotalEnergy(block=True))
minimised = process.getSystem()

Since we're working interactively, any time we query a running process we get back the _latest_ information that has been written to disk. This means that we can get an update on how things are progressing, then immediately carry on with what we were doing in our notebook. By passing `block=True`, as we do when we call `getTotalEnergy` above, we request that the process finishes running before returning a result. This means we get the _final_ energy, and the minimised system that is returned afterwards represents the _final_ snapshot that was saved.

Let's now re-run the simulation, instead using GROMACS as the MD engine.

In [ ]:
process = BSS.Process.Gromacs(system, protocol)

When the process is instantiated, BioSimSpace takes the system that was read from AMBER format files and converts it to GROMACS format ready for simulation. Let's take a look at the list of input files that were autogenerated for us:

In [ ]:
process.inputFiles()

Let's start the process running and, once again, wait for it to finish before getting the minimised system.

In [ ]:
process.start()
minimised = process.getSystem(block=True)

### Interactive molecular dynamics

The example in the previous section was finished almost as soon as it began. Let's run a more complicated equilibration protocol so that we can learn more about how to monitor processes interactively using BioSimSpace.

In [ ]:
protocol = BSS.Protocol.Equilibration(
    runtime=20 * BSS.Units.Time.picosecond,
    temperature_start=0 * BSS.Units.Temperature.kelvin,
    temperature_end=300 * BSS.Units.Temperature.kelvin,
    restraint="backbone",
)

This protocol will equlibrate a system for 20 picoseconds, while heating it from 0 to 300 Kelvin and restraining any atoms in the backbone of the molecule. Note that some of the parameters passed have units, e.g. the temperatures are in Kelvin. BioSimSpace has a built in type system for handling variables with units. The `BSS.Units` package provides a convenient way of declaring these, for example `10*BSS.Units.Temperature.kelvin` creaes an object of type `BSS.Types.Temperature` with a magnitude of 10 and unit of Kelvin. This allows the user to pass parameters with whatever unit they like. BioSimSpace will simply convert it to the correct unit for the chosen MD engine internally.

One again, we now need a `Process` in order to run our simulation. Exectute the cell below to initialise an AMBER process and start it immediately. Note that we pass in the minimised system from the last example, along with our new protocol.

In [ ]:
process = BSS.Process.Amber(minimised, protocol).start()

We can monitor the time, temperature, and energy as the process runs. If you run this multiple times using "CTRL+Return" you'll see the temperature slowly increasing.

In [ ]:
print(process.getTime(), process.getTemperature(), process.getTotalEnergy())

Since all of the values returned above are typed we can easily convert them to other units:

In [ ]:
print(
    process.getTime().picoseconds(),
    process.getTemperature().celsius(),
    process.getTotalEnergy().kj_per_mol(),
)

It's possible to query many other thermodynamic records. What's available depends on type of protocol and the MD package that is used to run the protocol. To get more information, run:

In [ ]:
help(process)

(Note that certain functionality is specific to the process in question, i.e. `BSS.Process.Amber` will have different options to `BSS.Process.Gromacs`, but, for the purposes of interoperability, there is a core set of functionality that is consistent across all `Process` classes, e.g. all classes implement a `getSystem` method.)

### Plotting time series data

As well as querying the most recent records we can also get a time series of results by passing the `time_series` keyword argument to any of the data record getter methods, e.g.

```python
# Get a time series of pressure records.
pressure = process.getPressure(time_series=True)
```

The `BSS.Notebook` package provides several useful tools that are available when working inside of a Jupyter notebook. One of these is the plot function, that allows us to create simple x/y plots of time-series data.

Let's grab the same record data as above and use it to make some graphs of the data.

In [ ]:
# Generate a plot of time vs temperature.
plot1 = BSS.Notebook.plot(
    process.getTime(time_series=True), process.getTemperature(time_series=True)
)

# Generate a plot of time vs energy.
plot2 = BSS.Notebook.plot(
    process.getTime(time_series=True), process.getTotalEnergy(time_series=True)
)

(Note that, by default, the axis labels axis labels are automatically generated from the types and units of the x and y data that are passed to the function.)

Re-run the cell using "CTRL+Return" to see the graphs update as the simulation progesses. (Occasionally, you might see a warning that the x and y data sets are mismatched in length, this is because the data was extracted before all records were written to disk.)

Being able to query a process in real time is an incredibly useful tool. This could enable us to check for convergence, or spot errors in the simulation. If you ever need to kill a running process (perhaps it was configured incorrectly), run:

```python
process.kill()
```

### Visualising the molecular system

Another useful tool that is available when working inside of a notebook is the `View` class that can be used to visualise the molecular system while a process is running. To create a `View` object we must attach it to a process (or a molecular system), e.g.:

In [ ]:
view = BSS.Notebook.View(process)

We can now visualise the system:

In [ ]:
view.system()

(If you see an empty view, try re-executing the cell.)

To only view a specific molecule:

In [ ]:
view.molecule(0)

To view a list of molecules:

In [ ]:
view.molecules([0, 5, 10])

If a particular view was of interest it can be reloaded as follows:

In [ ]:
# Reload the original view.
view.reload(0)

To save a specific view as a PDB file:

In [ ]:
view.savePDB("my_view.pdb", index=0)

### Reading and analysing trajectory data¶ 

The `BSS.Trajectory` package comes with a set of tools for reading and analysis trajectory files. Files can be loaded directly, or if supported, can be read from a running process.

For example, to get the trajectory from the process, run:

In [ ]:
traj = process.getTrajectory()

(If you get an error, then the trajectory file may be in the process of being written. Simply try again.)

To get the current number of frames:

In [ ]:
traj.nFrames()

In [ ]:
frames = traj.getFrames()

Specific frames can be extracted by passing a list of indices, e.g. the first and last:

In [ ]:
frames = traj.getFrames([0, -1])

Like most things in BioSimSpace, the `Trajectory` class is simply a wrapper around existing tools. Internally, trajectories are stored as an [MDTraj](http://mdtraj.org) object. This can be obtained, allowing the user direct access to the full power of MDTraj:

In [ ]:
mdtraj = traj.getTrajectory()
type(mdtraj)

Alternatively, a trajectory can be returned in [MDAnalysis](https://www.mdanalysis.org) format:

In [ ]:
mdanalysis = traj.getTrajectory(format="mdanalysis")
type(mdanalysis)

The `Trajectory` class also provides wrappers around some basic MDTraj analysis tools, allowing the user to compute quantities such as the root mean squared displacement (RMSD).

Let's measure the RMSD of the alanine-dipeptide molecule with a reference to its configuration in the first trajectory frame. To extract the alanine-dipeptide, we search the system for a residue named ALA. We'll also plot the RMSD for each frame of the trajectory.


In [ ]:
# Search the system for a residue named ALA. Since there is a single match, we take the first result.
residue = system.search("resname ALA").residues()[0]

# Get the indices of the atoms in the molecule, relative to the original system.
indices = [system.getIndex(x) for x in residue.getAtoms()]

# Compute the RMSD for each frame and plot the result.
BSS.Notebook.plot(
    y=process.getTrajectory().rmsd(frame=0, atoms=indices),
    xlabel="Frame",
    ylabel="RMSD",
)

Congratulations, you've just learned a lot about setting up and running molecular dynamics simulations with BioSimSpace. In the next notebook you'll learn how to write an interoperable workflow component called a _node_.